In [1]:
import contextily as cx
import xyzservices.providers as xyz
import matplotlib.pyplot as plt
import geopandas as gpd
import pyproj
import rasterio as rio
import geopandas as gpd #pip install geopandas descartes
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

In [2]:
def reproject_raster(in_path, out_path, dst_crs, bounds):

    # reproject raster to project crs
    with rio.open(in_path) as src:
        src_crs = src.crs
        print(f"Current projection: {src_crs}")
        transform, width, height = calculate_default_transform(src_crs, dst_crs, src.width, src.height, *bounds)
        kwargs = src.meta.copy()

        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height})

        with rio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.bilinear)
            print(f"New projection: {dst.crs}")
    return out_path

In [3]:
RASTER_DIR = "../raster_maps/"
areas = [
    'Centro',   
    'Oeste',
    'Sul1',
    'Sul2',
    'Norte1',
    'Norte2',
    'Leste1',
    'Leste2'
]

crs = pyproj.CRS({'proj': 'longlat', 'ellps': 'WGS84', 'no_defs': True})      


In [ ]:
# for area in areas:
#     db = gpd.read_file(f"../new_maps/{area}_modified/{area}_modified.shp")
#     db.to_crs({'init': 'epsg:3857'}, inplace=True)

#     w, s, e, n = db.total_bounds
#     reproject_raster(f"{RASTER_DIR}{area}.tiff", f"{RASTER_DIR}{area}_modified.tiff", crs, db.total_bounds)

In [6]:
JAWG_ACCESS_TOKEN="laGXnA7M9QfsEnrAWGVe1HSfgDF9ghAYeVn9bbTKLl7ClITmEUUCb1qKbQffeXnJ"
ESRI_ACCESS_TOKEN="AAPKa9fe3755a9f149e8957319837fc2a4e10P_k8HkiTW2z7MzIZgCVtDrMSMfNmRQrQe9wea2DLCTZfiDL-nTsQth4zmk2C9G1"

for area in areas:
    db = gpd.read_file(f"../new_maps/{area}_modified/{area}_modified.shp")
    db.to_crs({'init': 'epsg:3857'}, inplace=True)

    w, s, e, n = db.total_bounds
    # img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.CartoDB.Positron)
   
    # JAWG
    # img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.Jawg.Streets(accessToken=JAWG_ACCESS_TOKEN))
    # img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.Jawg.Sunny(accessToken=JAWG_ACCESS_TOKEN))
    # img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.Jawg.Light(accessToken=JAWG_ACCESS_TOKEN))
   
    # img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.OpenStreetMap.France)

    # ESRI
    # img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.Esri.WorldTopoMap(accessToken=ESRI_ACCESS_TOKEN))
    # img, ext = cx.bounds2raster(w, s, e, n, f"{RASTER_DIR}{area}.tiff", source=xyz.Esri.WorldStreetMap(accessToken=ESRI_ACCESS_TOKEN))

    reproject_raster(f"{RASTER_DIR}{area}.tiff", f"{RASTER_DIR}{area}_modified.tiff", crs, db.total_bounds)


/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/contextily/tile.py:581: UserWarning: The inferred zoom level of 14 is not valid for the current tile provider (valid zooms: 1 - 11).
  warnings.warn(msg)


Current projection: EPSG:3857
New projection: GEOGCS["unknown",DATUM["Unknown based on WGS84 ellipsoid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]


/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/contextily/tile.py:581: UserWarning: The inferred zoom level of 13 is not valid for the current tile provider (valid zooms: 1 - 11).
  warnings.warn(msg)


Current projection: EPSG:3857
New projection: GEOGCS["unknown",DATUM["Unknown based on WGS84 ellipsoid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]


/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/contextily/tile.py:581: UserWarning: The inferred zoom level of 13 is not valid for the current tile provider (valid zooms: 1 - 11).
  warnings.warn(msg)


...

Current projection: EPSG:3857
New projection: GEOGCS["unknown",DATUM["Unknown based on WGS84 ellipsoid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]
Current projection: EPSG:3857


/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/contextily/tile.py:581: UserWarning: The inferred zoom level of 13 is not valid for the current tile provider (valid zooms: 1 - 11).
  warnings.warn(msg)


New projection: GEOGCS["unknown",DATUM["Unknown based on WGS84 ellipsoid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]


/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/milenacsilva/Área de Trabalho/ic/env/lib/python3.9/site-packages/contextily/tile.py:581: UserWarning: The inferred zoom level of 13 is not valid for the current tile provider (valid zooms: 1 - 11).
  warnings.warn(msg)


Current projection: EPSG:3857
New projection: GEOGCS["unknown",DATUM["Unknown based on WGS84 ellipsoid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]
